


REQUIREMENTS 
Use Linear Regression to predict Boston housing prices.
Compare the results to other ML algorithms.

REFERENCE
https://spark.apache.org/docs/latest/ml-classification-regression.html
https://spark.apache.org/docs/latest/api/python/_modules/pyspark/ml/regression.html


    

This notebook is designed to run in a IBM Watson Studio default runtime (NOT the Watson Studio Apache Spark Runtime as the default runtime with 1 vCPU is free of charge). 
Therefore, we install Apache Spark in local mode for test purposes only. 
Don't use it in production.

If running outside Watson Studio, this should work as well. 
In case you are running in an Apache Spark context outside Watson Studio, remove the Apache Spark setup in the first notebook cells.

In [1]:
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown('# <span style="color:red">'+string+'</span>'))


if ('sc' in locals() or 'sc' in globals()):
    printmd('<<<<<!!!!! It seems that you are running in a IBM Watson Studio Apache Spark Notebook. Please run it in an IBM Watson Studio Default Runtime (without Apache Spark) !!!!!>>>>>')


In [2]:
!pip install pyspark==2.4.5

     |████████████████████████████████| 217.8MB 34.5MB/s eta 0:00:01  |▎                               | 1.7MB 9.1MB/s eta 0:00:24     |███████████████████▏            | 130.4MB 37.4MB/s eta 0:00:03     |████████████████████████▊       | 168.3MB 30.1MB/s eta 0:00:02
     |████████████████████████████████| 204kB 30.7MB/s eta 0:00:01
  Stored in directory: /home/dsxuser/.cache/pip/wheels/bf/db/04/61d66a5939364e756eb1c1be4ec5bdce6e04047fc7929a3c3c
Successfully built pyspark


In [3]:
try:
    from pyspark import SparkContext, SparkConf
    from pyspark.sql import SparkSession
except ImportError as e:
    printmd('<<<<<!!!!! Please restart your kernel after installing Apache Spark !!!!!>>>>>')

STEP1. LOAD THE DATA

In [4]:
# The code was removed by Watson Studio for sharing.

In [5]:

# Upload the Boston.csv file from: https://www.kaggle.com/puxama/bostoncsv
### YOUR CODE HERE ###

# Create pandas dataframe from the uploaded Boston.csv file:
house_pdf = pd.read_csv(body)
house_pdf.head()


,Unnamed: 0,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv
0,1,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,2,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,3,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,4,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,5,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


In [6]:
# The input data set is from a Kaggle competition: Housing Values in Suburbs of Boston. 

# For each house observation, we have the following information:
# CRIM — per capita crime rate by town.
# ZN — proportion of residential land zoned for lots over 25,000 sq.ft.
# INDUS — proportion of non-retail business acres per town.
# CHAS — Charles River dummy variable (= 1 if tract bounds river; 0 otherwise).
# NOX — nitrogen oxides concentration (parts per 10 million).
# RM — average number of rooms per dwelling.
# AGE — proportion of owner-occupied units built prior to 1940.
# DIS — weighted mean of distances to five Boston employment centres.
# RAD — index of accessibility to radial highways.
# TAX — full-value property-tax rate per $10,000.
# PTRATIO — pupil-teacher ratio by town.
# BLACK — 1000(Bk — 0.63)² where Bk is the proportion of blacks by town.
# LSTAT — lower status of the population (percent).
# MEDV — median value of owner-occupied homes in $1000s. This is the target variable.

# Based on this input data set, we'll try to determine a model that can predict the median value of a given house in the area.

In [7]:

#  Create a Spark SQL context to work with:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext
sc= SparkContext()
sqlContext = SQLContext(sc)

# Convert the previous pandas dataframe to a pyspark dataframe with Boston housing information:
house_df = sqlContext.createDataFrame(house_pdf)
house_df.take(1)


[Row(Unnamed: 0=1, crim=0.00632, zn=18.0, indus=2.31, chas=0, nox=0.5379999999999999, rm=6.575, age=65.2, dis=4.09, rad=1, tax=296, ptratio=15.3, black=396.9, lstat=4.98, medv=24.0)]

STEP2. EXPLORE THE DATA

In [8]:

# Print the schema in a tree format:
house_df.cache()
house_df.printSchema()


root
 |-- Unnamed: 0: long (nullable = true)
 |-- crim: double (nullable = true)
 |-- zn: double (nullable = true)
 |-- indus: double (nullable = true)
 |-- chas: long (nullable = true)
 |-- nox: double (nullable = true)
 |-- rm: double (nullable = true)
 |-- age: double (nullable = true)
 |-- dis: double (nullable = true)
 |-- rad: long (nullable = true)
 |-- tax: long (nullable = true)
 |-- ptratio: double (nullable = true)
 |-- black: double (nullable = true)
 |-- lstat: double (nullable = true)
 |-- medv: double (nullable = true)



In [9]:

# Perform descriptive analytics
house_df.describe().toPandas().transpose()


,0,1,2,3,4
summary,count,mean,stddev,min,max
Unnamed: 0,506,253.5,146.2138844296259,1,506
crim,506,3.6135235573122535,8.601545105332491,0.00632,88.9762
zn,506,11.363636363636363,23.32245299451514,0.0,100.0
indus,506,11.136778656126504,6.860352940897589,0.46,27.74
chas,506,0.0691699604743083,0.2539940413404101,0,1
nox,506,0.5546950592885372,0.11587767566755584,0.385,0.871
rm,506,6.284634387351788,0.7026171434153232,3.5610000000000004,8.78
age,506,68.57490118577078,28.148861406903595,2.9,100.0
dis,506,3.795042687747034,2.10571012662761,1.1296,12.1265



PLOT THE DATA USING A SCATTER MATRIX

Scatter matrix is a great way to roughly determine if we have a linear correlation between multiple independent variables.
A scatter matrix (pairs plot) compactly plots all the numeric variables we have in a dataset against each other.


In [10]:

# Sample the data to avoid huge inputs for scatter matrix plot:
import pandas as pd
numeric_features = [t[0] for t in house_df.dtypes if t[1] == 'int' or t[1] == 'double']
sampled_data = house_df.select(numeric_features).sample(False, 0.8).toPandas()

# Plot the corresponding scatter matrix:
from pandas.plotting import scatter_matrix
axs = scatter_matrix(sampled_data, figsize=(10, 10))
n = len(sampled_data.columns)
for i in range(n):
    v = axs[i, 0]
    v.yaxis.label.set_rotation(0)
    v.yaxis.label.set_ha('right')
    v.set_yticks(())
    h = axs[n-1, i]
    h.xaxis.label.set_rotation(90)
    h.set_xticks(())    



The diagonal of the scatter matrix shows the distribution of all the variables in our test data.
In the other cells of the plot matrix, we have the scatterplots (i.e. correlation plot) of each variable combination in our dataframe.

But it is hard to see if we have a linear correlation between the variables. 
Let’s compute the correlations between the independent variables and the target variable.


In [11]:

house_df.take(1)

import six
for i in house_df.columns: # Take each column of the dataset:
    if not( isinstance(house_df.select(i).take(1)[0][0], six.string_types)): # If column values are not Strings:
        print( "Correlation to medv (house median values) for ", i, house_df.stat.corr('medv', i)) # Check for a corr between the column and the target var (house prices). 
        

Correlation to medv (house median values) for  Unnamed: 0 -0.22660364293533927
Correlation to medv (house median values) for  crim -0.38830460858681154
Correlation to medv (house median values) for  zn 0.3604453424505433
Correlation to medv (house median values) for  indus -0.4837251600283728
Correlation to medv (house median values) for  chas 0.1752601771902987
Correlation to medv (house median values) for  nox -0.42732077237328203
Correlation to medv (house median values) for  rm 0.69535994707154
Correlation to medv (house median values) for  age -0.3769545650045961
Correlation to medv (house median values) for  dis 0.249928734085904
Correlation to medv (house median values) for  rad -0.38162623063977735
Correlation to medv (house median values) for  tax -0.46853593356776674
Correlation to medv (house median values) for  ptratio -0.5077866855375622
Correlation to medv (house median values) for  black 0.3334608196570661
Correlation to medv (house median values) for  lstat -0.737662726


The correlation coefficient ranges from –1 to 1. 

When it is close to 1, it means that there is a strong positive correlation; for example, the median value tends to go up when the number of rooms goes up. 
When the coefficient is close to –1, it means that there is a strong negative correlation; the median value tends to go down when the percentage of the lower status of the population goes up. 
Coefficients close to zero mean that there is no linear correlation.



We are going to keep all the variables, for now.
But we prepare the data for Machine Learning algorithms, fitting it in only two columns: features and label(“medv”).
    

In [12]:

from pyspark.ml.feature import VectorAssembler
vectorAssembler = VectorAssembler(inputCols = ['crim', 'zn', 'indus', 'chas', 'nox', 'rm', 'age', 'dis', 'rad', 'tax', 'ptratio', 'black', 'lstat'], outputCol = 'features')
vhouse_df = vectorAssembler.transform(house_df)
vhouse_df = vhouse_df.select(['features', 'medv'])
vhouse_df.show(3)


+--------------------+----+
|            features|medv|
+--------------------+----+
|[0.00632,18.0,2.3...|24.0|
|[0.02731,0.0,7.07...|21.6|
|[0.02729,0.0,7.07...|34.7|
+--------------------+----+
only showing top 3 rows




Split the data in training and test data sets


In [28]:
splits = vhouse_df.randomSplit([0.7, 0.3])
train_df = splits[0]
test_df = splits[1]


STEP3. LINEAR REGRESSION IMPLEMENTATION WITH SPARK ML

Linear Regression Parameters:

If your linear model contains many predictor variables X or if these variables are correlated, 
the parameter estimates have large variance (the model overfits the training data), thus making the model unreliable.

To counter this, you can use regularization: a technique allowing to decrease this variance at the cost of introducing some bias. 
Finding a good bias-variance trade-off allows to minimize the model's total error.

There are three popular regularization techniques, each of them aiming at decreasing the size of the coefficients:
  Lasso Regression, which penalizes the sum of absolute values of the coefficients (L1 penalty).
  Ridge Regression, which penalizes the sum of squared coefficients (L2 penalty).
  Elastic Net, a convex combination of Ridge and Lasso meant to take the best from both.
The size of the respective penalty terms can be tuned via cross-validation to find the model's best fit.

Example of linear regression parameters choices:

regParam=0.3        This is the penalty strength parameter: the AMOUNT of shrinkage in model parameters number. It is usually noted with lambda λ. λ >= 0.

elasticNetParam=0.8 This is the ratio between L1 and L2 regularization TECHNIQUES. It is usually nothed with alpha (α). And α is in [0, 1] range. 
                    For alpha = 0, the penalty is an L2 or Ridge penalty. 
                    For alpha = 1, the penalty is an L1 or Lasso penalty.

In [29]:

from pyspark.ml.regression import LinearRegression

# Reference: https://spark.apache.org/docs/latest/ml-classification-regression.html
# Make sure to also specify featuresCol and labelCol
# lr = ### YOUR CODE HERE ### 


In [30]:

# Train the model on the training dataset:
lr_model = lr.fit(train_df)

# Print the coefficients and intercept for linear regression
print("Coefficients: " + str(lr_model.coefficients))
print("Intercept: " + str(lr_model.intercept))


Coefficients: [-0.02843385073730485,0.006987958941808668,-0.006018303837825146,2.766194104992275,-1.7764473377226828,4.110793844028774,0.0,-0.5033371587694981,0.0,-0.000521006185267343,-0.7457130584285585,0.010967073089205345,-0.5824436825880731]
Intercept: 16.888442985401326


Resulting equation: y = cf1 * x1 + cf2 * x2 + ... + cfn * xn + intercept


Now, summarize the model over the training set and print out some metrics:
    

In [31]:

trainingSummary = lr_model.summary
print("Root Mean Squared Error (RMSE) on training data: %f" % trainingSummary.rootMeanSquaredError) 
print("R Squared (R2) on training data: %f" % trainingSummary.r2)


Root Mean Squared Error (RMSE) on training data: 4.796013
R Squared (R2) on training data: 0.723605



RMSE measures the differences between the values predicted by the model and the actual values. 
However, RMSE alone is meaningless until we compare with the actual “medv” value, such as mean, min and max. 
After such comparison, our RMSE looks pretty good.


In [32]:

train_df.describe().show()


+-------+------------------+
|summary|              medv|
+-------+------------------+
|  count|               362|
|   mean|22.562430939226516|
| stddev| 9.135148560725185|
|    min|               5.0|
|    max|              50.0|
+-------+------------------+




R squared at 0.74 indicates that approximate 74% of the variability in “medv” can be explained using our model. 
It is not bad. However, we must be cautious because the performance on the training set may not be a good approximation of the performance on the test data set.


In [33]:

# Make predictions on the test data:
lr_predictions = lr_model.transform(test_df)
lr_predictions.select("prediction","medv","features").show(5)

# Evaluate predictions on test data:
from pyspark.ml.evaluation import RegressionEvaluator
lr_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="medv",metricName="r2")
print("R Squared (R2) on test data = %g" % lr_evaluator.evaluate(lr_predictions))


+------------------+----+--------------------+
|        prediction|medv|            features|
+------------------+----+--------------------+
|16.933581107514826|18.9|[0.0136,75.0,4.0,...|
| 41.42348741182059|50.0|[0.01500999999999...|
| 38.77352246638074|50.0|[0.02009,95.0,2.6...|
| 36.22357435258677|42.3|[0.02176999999999...|
| 31.01318747473683|31.1|[0.02187,60.0,2.9...|
+------------------+----+--------------------+
only showing top 5 rows

R Squared (R2) on test data = 0.682175


In [34]:

test_result = lr_model.evaluate(test_df)
print("Root Mean Squared Error (RMSE) on test data = %g" % test_result.rootMeanSquaredError)


Root Mean Squared Error (RMSE) on test data = 5.27127



Sure enough, we achieved worse RMSE and R squared on the test set.

Residuals

The difference between the observed value of the dependent variable (y) and the predicted value (ŷ) is called the residual (e). 
Each data point has one residual: e = y - ŷ. Both the sum and the mean of the residuals are equal to zero.
Ideally all residuals should be small and unstructured (random aspect, no patterns, just vary around 0) 

Reference: http://www.unige.ch/ses/sococ/cl////stat/action/analyse_residuals0.html?


In [35]:
trainingSummary.residuals.show()

+--------------------+
|           residuals|
+--------------------+
|  -6.902870284999899|
|  1.7532368471147741|
| -5.3865068819629585|
|  0.7128558346660654|
|  5.1098134537748585|
|  12.053323384827635|
| 0.30757694362829824|
| -1.3034518517958986|
| -3.2130995980123096|
|   9.311248144988383|
|  2.9490699014573636|
|  1.8584828656162529|
| -3.1529447043191325|
|   6.535236333726118|
|-0.20552527797233466|
| -1.5919952668108195|
| -10.348266220554464|
|  -3.983768157477524|
|  -3.936764631984687|
|  1.7293042154141212|
+--------------------+
only showing top 20 rows



In [36]:

# Print the objective per iteration: the objective per iteration is to minimize the cost function (the error of the model).
# 
# To define and measure the error of our model we define the cost function as the sum of the squares of the residuals. 
# We should observe that the cost function (the error of the model) decreases with each iteration and converges after a number of iterations.
# Reference: https://towardsdatascience.com/linear-regression-using-python-b136c91bf0a2
# 
print("objectiveHistory: %s" % str(trainingSummary.objectiveHistory))
print("numIterations: %d" % trainingSummary.totalIterations)


objectiveHistory: [0.5, 0.43245577500547494, 0.2355007629342825, 0.21139271977337173, 0.18068237997375186, 0.17788288174175415, 0.17682238630551858, 0.17589667986395252, 0.17491822484378208, 0.1741280827040414, 0.1740053945759089]
numIterations: 11


LINEAR REGRESSION vs DECISION TREE REGRESSION vs GRADIENT BOOSTING TREE REGRESSION

LINEAR REGRESSION 
It is used to predict continuous outputs where there is a linear relationship between the features of the dataset and the output variable. 
It is used for regression problems where you are trying to predict something with infinite possible answers such as the price of a house.

When to use linear regression
It is appropriate for datasets with a linear relationship between the features and the output variable. 
POLYNOMIAL REGRESSION can also be used when there is a non-linear relationship between the features and the output.

DECISION TREES
They can be used for either classification or regression problems and are useful for complex datasets. 
They work by splitting the dataset, in a tree-like structure, into smaller and smaller subsets 
and then make predictions based on what subset a new example would fall into.

When to use decision trees
They are useful when there are complex relationships between the features and the output variables. 
They also work well compared to other algorithms when there are missing features, when there is a mix of categorical and numerical features 
and when there is a big difference in the scale of features.

DECISION TREE vs RANDOM FOREST vs GRADIENT BOOSTING MACHINES
A decision tree is a simple, decision making-diagram. 
Random forests are a large number of trees, combined (using averages or "majority rules") at the end of the process. 
Gradient boosting machines also combine decision trees, but start the combining process at the beginning, instead of at the end.

REFERENCE: 
https://mlcorner.com/linear-regression-vs-decision-trees/
https://www.datasciencecentral.com/profiles/blogs/decision-tree-vs-random-forest-vs-boosted-trees-explained#:~:text=In%20a%20nutshell%3A,instead%20of%20at%20the%20end


STEP4. DECISION TREE REGRESSION WITH SPARK ML


In [37]:

from pyspark.ml.regression import DecisionTreeRegressor

# Reference: https://spark.apache.org/docs/latest/ml-classification-regression.html
# Make sure to specify featuresCol and labelCol
# dt = ### YOUR CODE HERE ###

# Train the model:
dt_model = dt.fit(train_df)

# Make predictions on the test data:
dt_predictions = dt_model.transform(test_df)

# Evaluate prediction accuracy:
dt_evaluator = RegressionEvaluator(
    labelCol="medv", predictionCol="prediction", metricName="rmse")
rmse = dt_evaluator.evaluate(dt_predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

dt_evaluator_r2 = RegressionEvaluator(
    labelCol="medv", predictionCol="prediction", metricName="r2")
r2 = dt_evaluator_r2.evaluate(dt_predictions)
print("R Squared (R2) on test data = %g" % r2)


Root Mean Squared Error (RMSE) on test data = 3.75437
R Squared (R2) on test data = 0.838776



Feature Importances 

They are assigned scores to input features showing the relative importance of each feature when making a prediction.
Feature importances are useful for feature selection: 
    the process of identifying and selecting a subset of input variables that are most relevant to the target variable.

Top reasons to use feature selection are:
    It enables the machine learning algorithm to train faster.
    It reduces the complexity of a model and makes it easier to interpret.
    It improves the accuracy of a model if the right subset is chosen.
    It reduces overfitting.
    
Reference: https://www.analyticsvidhya.com/blog/2016/12/introduction-to-feature-selection-methods-with-an-example-or-how-to-select-the-right-variables/
        

In [38]:

# Feature importances:
dt_model.featureImportances 


SparseVector(13, {0: 0.0584, 4: 0.0104, 5: 0.5725, 6: 0.0009, 7: 0.0858, 9: 0.0042, 10: 0.0555, 11: 0.0042, 12: 0.2081})

In [39]:

# And the features are:
house_df.take(1)


[Row(Unnamed: 0=1, crim=0.00632, zn=18.0, indus=2.31, chas=0, nox=0.5379999999999999, rm=6.575, age=65.2, dis=4.09, rad=1, tax=296, ptratio=15.3, black=396.9, lstat=4.98, medv=24.0)]

Apparently, the number of rooms is the most important feature to predict the house median price in our data.


STEP5. GRADIENT BOOSTED TREE REGRESSION WITH SPARK ML

REMEMBER:

DECISION TREE vs RANDOM FOREST vs GRADIENT BOOSTING MACHINES
A decision tree is a simple, decision making-diagram. 
Random forests are a large number of trees, combined (using averages or "majority rules") at the end of the process. 
Gradient boosting machines also combine decision trees, but start the combining process at the beginning, instead of at the end.

Reference: 
https://www.datasciencecentral.com/profiles/blogs/decision-tree-vs-random-forest-vs-boosted-trees-explained#:~:text=In%20a%20nutshell%3A,instead%20of%20at%20the%20end


In [40]:

from pyspark.ml.regression import GBTRegressor

# Reference: https://spark.apache.org/docs/latest/ml-classification-regression.html
# Make sure to also specify featuresCol and labelCol
# gbt = ### YOUR CODE HERE ###

# Train the model:
gbt_model = gbt.fit(train_df)

# Make predictions on the test data:
gbt_predictions = gbt_model.transform(test_df)
gbt_predictions.select('prediction', 'medv', 'features').show(5)


+------------------+----+--------------------+
|        prediction|medv|            features|
+------------------+----+--------------------+
| 15.34769474911794|18.9|[0.0136,75.0,4.0,...|
| 46.62320773492755|50.0|[0.01500999999999...|
| 47.10060831831482|50.0|[0.02009,95.0,2.6...|
|47.221387758214696|42.3|[0.02176999999999...|
| 30.07703851167657|31.1|[0.02187,60.0,2.9...|
+------------------+----+--------------------+
only showing top 5 rows



In [41]:

# Evaluate the predictions:

gbt_evaluator = RegressionEvaluator(
    labelCol="medv", predictionCol="prediction", metricName="rmse")
rmse = gbt_evaluator.evaluate(gbt_predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

gbt_evaluator_r2 = RegressionEvaluator(
    labelCol="medv", predictionCol="prediction", metricName="r2")
r2 = gbt_evaluator_r2.evaluate(gbt_predictions)
print("R Squared (R2) on test data = %g" % r2)


Root Mean Squared Error (RMSE) on test data = 3.50416
R Squared (R2) on test data = 0.859549


CONCLUSION: 
    Gradient-boosted tree regression performed the best on our data.

REFERENCE CASE STUDY: https://towardsdatascience.com/building-a-linear-regression-with-pyspark-and-mllib-d065c3ba246a

OPTIONAL HOMEWORK:

Predict Boston housing prices using a Random Forest algorithm.

REMEMBER:

DECISION TREE vs RANDOM FOREST vs GRADIENT BOOSTING MACHINES 
A decision tree is a simple, decision making-diagram. 
Random forests are a large number of trees, combined (using averages or "majority rules") at the end of the process. 
Gradient boosting machines also combine decision trees, but start the combining process at the beginning, instead of at the end.

REFERENCE: 
https://spark.apache.org/docs/latest/ml-classification-regression.html
https://spark.apache.org/docs/latest/api/python/_modules/pyspark/ml/regression.html